In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from pydantic import BaseModel
from agents import Agent, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered, RunContextWrapper, Runner, TResponseInputItem, input_guardrail, output_guardrail

class MathHomeWorkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str
    answer: str

guardrail_agent = Agent(
    name = "Guardrail Agent",
    instructions="Check if the input is math homework and provide a solution.",
    output_type=MathHomeWorkOutput,
)

@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input_item: str | list[TResponseInputItem]) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info= result.final_output,
        tripwire_triggered= result.final_output.is_math_homework,
    )

agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
)


try:
    result = await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
    print("Guardrail didn't trip - this is unexpected")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

     
     

TypeError: cannot pickle '_contextvars.Context' object

In [11]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
    output_guardrail,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)

class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)


@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        # tripwire_triggered=False #result.final_output.is_math_homework,
        tripwire_triggered=result.final_output.is_math_homework,
    )
     
agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
)


# This should trip the guardrail

try:
    result = await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
    print("Guardrail didn't trip - this is unexpected")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

Math homework guardrail tripped


In [9]:
try:
    result = await Runner.run(agent, "Hello")
    print(result.final_output)

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")

Hi there! How can I assist you today?


In [17]:
### output 


class MessageOutput(BaseModel):
    response: str

class MathOutput(BaseModel):
    is_math: bool
    reasoning: str

guardrail_agent2 = Agent(
    name="Guardrail check",
    instructions="Check if the output includes any math.",
    output_type=MathOutput,
)

@output_guardrail
async def math_guardrail2(
    ctx: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent2, output.response, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )
     

agent2 = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[math_guardrail2],
    output_type=MessageOutput,
)

# This should trip the guardrail
try:
    await Runner.run(agent2, "Hello, can you help me solve for x: 2x + 3 = 11?")
    print("Guardrail didn't trip - this is unexpected")

except OutputGuardrailTripwireTriggered:
    print("Math output guardrail tripped")

Math output guardrail tripped


In [19]:

import asyncio
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrail,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
)

# Define the output model for the guardrail agent
class PIAICRelevanceOutput(BaseModel):
    is_piaic_relevant: bool
    reasoning: str

# Create the guardrail agent to check if input is PIAIC-related
guardrail_agent = Agent(
    name="PIAIC_Relevance_Check",
    instructions=(
        "You are a guardrail agent that checks if the user's input is related to PIAIC (Presidential Initiative for Artificial Intelligence and Computing) topics, "
        "such as Artificial Intelligence, Cloud Native Computing, Blockchain, Internet of Things (IoT), or other PIAIC courses. "
        "Determine if the input is relevant to PIAIC. "
        "Return a structured output with 'is_piaic_relevant' as a boolean and 'reasoning' explaining your decision."
    ),
    output_type=PIAICRelevanceOutput,
)

# Define the input guardrail function
async def piaic_relevance_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str | list,
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)
    final_output = result.final_output_as(PIAICRelevanceOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_piaic_relevant,
    )

# Create the main PIAIC agent
piaic_agent = Agent(
    name="PIAIC_Assistant",
    instructions=(
        "You are a helpful assistant for PIAIC-related questions. "
        "Answer questions about PIAIC courses, such as AI, Cloud Native Computing, Blockchain, IoT, or other PIAIC initiatives. "
        "Provide accurate and concise information."
    ),
    input_guardrails=[InputGuardrail(guardrail_function=piaic_relevance_guardrail)],
)


try:
    result = await Runner.run(piaic_agent, "What is the curriculum for PIAIC's AI course?")
    print("Response:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("Guardrail tripped: Input is not PIAIC-related.")





Response: The PIAIC Artificial Intelligence course is structured into four quarters, each covering different aspects of AI:

**Quarter 1: AI Foundations**
- Introduction to AI and its applications
- Basics of Python programming and libraries
- Data manipulation and analysis using Pandas and NumPy
- Introduction to data visualization with Matplotlib and Seaborn

**Quarter 2: Advanced Python & Machine Learning Basics**
- Advanced Python techniques
- Object-oriented and functional programming
- Introduction to machine learning principles
- Supervised and unsupervised learning techniques
- Implementation of ML algorithms using Scikit-learn

**Quarter 3: Deep Learning and Computer Vision**
- Fundamentals of deep learning
- Neural networks and their architectures
- Convolutional neural networks (CNNs) for image processing
- Hands-on projects using TensorFlow and Keras

**Quarter 4: NLP, AI Specializations, and Deployment**
- Natural Language Processing (NLP) techniques
- Transformer models a

In [21]:
# Test with non-PIAIC input
try:
    result = await Runner.run(piaic_agent, "How do I bake a chocolate cake")
    print("Response:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("Guardrail tripped: Input is not PIAIC-related.")

Guardrail tripped: Input is not PIAIC-related.


In [22]:
## INPUT & OUTPUT Gradrail


import asyncio
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrail,
    InputGuardrailTripwireTriggered,
    OutputGuardrail,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
)

# Define the output model for the guardrail agents
class PIAICRelevanceOutput(BaseModel):
    is_piaic_relevant: bool
    reasoning: str

# Create the input guardrail agent to check if input is PIAIC-related
input_guardrail_agent = Agent(
    name="PIAIC_Input_Relevance_Check",
    instructions=(
        "You are a guardrail agent that checks if the user's input is related to PIAIC (Presidential Initiative for Artificial Intelligence and Computing) topics, "
        "such as Artificial Intelligence, Cloud Native Computing, Blockchain, Internet of Things (IoT), or other PIAIC courses. "
        "Determine if the input is relevant to PIAIC. "
        "Return a structured output with 'is_piaic_relevant' as a boolean and 'reasoning' explaining your decision."
    ),
    output_type=PIAICRelevanceOutput,
)

# Create the output guardrail agent to check if output is PIAIC-related
output_guardrail_agent = Agent(
    name="PIAIC_Output_Relevance_Check",
    instructions=(
        "You are a guardrail agent that checks if the agent's response is related to PIAIC (Presidential Initiative for Artificial Intelligence and Computing) topics, "
        "such as Artificial Intelligence, Cloud Native Computing, Blockchain, Internet of Things (IoT), or other PIAIC courses. "
        "Determine if the response content is relevant to PIAIC. "
        "Return a structured output with 'is_piaic_relevant' as a boolean and 'reasoning' explaining your decision."
    ),
    output_type=PIAICRelevanceOutput,
)

# Define the input guardrail function
async def piaic_input_relevance_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    input: str | list,
) -> GuardrailFunctionOutput:
    result = await Runner.run(input_guardrail_agent, input, context=ctx.context)
    final_output = result.final_output_as(PIAICRelevanceOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_piaic_relevant,
    )

# Define the output guardrail function
async def piaic_output_relevance_guardrail(
    ctx: RunContextWrapper[None],
    agent: Agent,
    output: str | list,
) -> GuardrailFunctionOutput:
    result = await Runner.run(output_guardrail_agent, output, context=ctx.context)
    final_output = result.final_output_as(PIAICRelevanceOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_piaic_relevant,
    )

# Create the main PIAIC agent with both input and output guardrails
piaic_agent = Agent(
    name="PIAIC_Assistant",
    instructions=(
        "You are a helpful assistant for PIAIC-related questions. "
        "Answer questions about PIAIC courses, such as AI, Cloud Native Computing, Blockchain, IoT, or other PIAIC initiatives. "
        "Provide accurate and concise information."
    ),
    input_guardrails=[InputGuardrail(guardrail_function=piaic_input_relevance_guardrail)],
    output_guardrails=[OutputGuardrail(guardrail_function=piaic_output_relevance_guardrail)],
)


try:
    result = await Runner.run(piaic_agent, "What is the curriculum for PIAIC's AI course?")
    print("Response:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("Input Guardrail tripped: Input is not PIAIC-related.")
except OutputGuardrailTripwireTriggered as e:
    print("Output Guardrail tripped: Response is not PIAIC-related.")

Response: The PIAIC AI curriculum is structured to provide a comprehensive understanding of artificial intelligence. It is divided into several modules, typically including:

1. **Introduction to AI and Deep Learning**
   - Overview of AI and its applications
   - Basics of deep learning and neural networks
   - Setup of development environments

2. **Python for AI**
   - Python programming fundamentals
   - Libraries like NumPy, Pandas, Matplotlib

3. **Data Science Essentials**
   - Data manipulation and analysis
   - Data visualization techniques

4. **Machine Learning Basics**
   - Supervised and unsupervised learning
   - Algorithms such as regression, classification, clustering

5. **Advanced Deep Learning Techniques**
   - Convolutional Neural Networks (CNNs)
   - Recurrent Neural Networks (RNNs)
   - Generative models

6. **AI for Computer Vision**
   - Image processing and interpretation
   - Object detection and recognition

7. **AI for Natural Language Processing (NLP)**
   

In [24]:
# Test with non-PIAIC input
try:
    result = await Runner.run(piaic_agent, "How do I bake a chocolate cake?")
    print("Response:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("Input Guardrail tripped: Input is not PIAIC-related.")
except OutputGuardrailTripwireTriggered as e:
    print("Output Guardrail tripped: Response is not PIAIC-related.")

     

Input Guardrail tripped: Input is not PIAIC-related.


In [26]:
# Test with non-PIAIC input
try:
    result = await Runner.run(piaic_agent, "tell me about piaic founder, tell me about his current job title. he has left PIAIC as per my information. how he can cook chicken.")
    print("Response:", result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("Input Guardrail tripped: Input is not PIAIC-related.")
except OutputGuardrailTripwireTriggered as e:
    print("Output Guardrail tripped: Response is not PIAIC-related.")

Response: The founder of the Presidential Initiative for Artificial Intelligence and Computing (PIAIC) is Zia Khan. He is an influential figure in technology education in Pakistan. As of my latest information, he is involved with Panacloud, where he focuses on advancing technology training and education.

Regarding cooking chicken, here’s a simple way he might approach it:

1. **Ingredients**: Chicken pieces, salt, pepper, olive oil, garlic, lemon juice, and herbs (like rosemary or thyme).

2. **Preparation**:
   - Season the chicken with salt and pepper.
   - Heat olive oil in a pan over medium heat.
   - Add minced garlic, cook until fragrant.
   - Place chicken in the pan, cook until browned on both sides.
   - Add lemon juice and herbs, then cover and simmer until fully cooked.

This is a basic method, but there are countless variations and recipes to explore!
